# Spoint tutorial: Deconvolution of cell types composition on human brain Visium dataset

July 2023

Dataset: Visium slices of human dorsolateral prefrontal cortex ([here](https://zenodo.org/record/8167488)).

In [51]:
import sys
sys.path.append('/data/wzh/SPACEL-l')

In [52]:
from KanCell.setting import set_environ_seed
set_environ_seed()
from KanCell import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


## Load spatial transcriptomics data

The input data are anndata objects stored raw counts for scRNA-seq and ST. The scRNA-seq anndata must have cell type annotation in `.obs`.

In [53]:
sc_ad = sc.read_h5ad('/data/KanCell-main/docs/data/starmap/starmap_sc_rna.h5ad')
st_ad = sc.read_h5ad('/data/KanCell-main/docs/data/starmap/starmap_spatial.h5ad')
sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [54]:
sc_ad

AnnData object with n_obs × n_vars = 14249 × 34041
    obs: 'celltype', 'n_genes'
    var: 'n_cells'

## Initialize the Spoint model

In this step, we initialize the Spoint model using anndata objects for scRNA-seq and ST as input. The `celltype_key` parameter represents the column name of the cell type annotation in the `.obs` attribute of the scRNA-seq anndata object. The `sm_size` parameter controls the number of simulated spots, and it is important to have a sufficient `sm_size` for accurate prediction. However, it should be noted that increasing the `sm_size` will also increase the simulation and training time. In general, we recommend setting `sm_size` to a value greater than 100,000.

In [55]:
import os,sys
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
celltype
Excitatory L2/3     200
Excitatory L6       200
Excitatory L5       197
Excitatory L4       190
Inhibitory Other    185
Inhibitory Sst      178
Inhibitory Pvalb    174
Inhibitory Vip      170
Astro               156
Other               120
Endo                107
Olig                 97
Smc                  94
Micro                57
Neuron Other         51
Name: count, dtype: int64
### Used gene numbers: 791
### Initializing sample probability
### 

## Training the Spoint model

Here, we train the model to obtain the optimal model for cell type deconvolution. The `max_steps` parameter represents the maximum number of steps in the training process. If the `early_stop` parameter is set to `True`, the model will stop training before reaching the maximum number of steps if the model has converged.

In [56]:

import os,sys
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
spoint_model.train(max_steps=5000, batch_size=1024)

/root/anaconda3/envs/SPACEL/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]


Epoch 100/100: 100%|██████████| 100/100 [01:18<00:00,  1.29it/s, loss=1.12e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [01:18<00:00,  1.27it/s, loss=1.12e+03, v_num=1]


Step 359: Test inference loss=-0.947:   7%|▋         | 359/5000 [01:36<16:08,  4.79it/s]

## Visualization results

Then, we utilize the trained model to predict the cell type composition of each spot in the spatial transcriptomics data. This prediction will generate a `DataFrame` object, where each row corresponds to a spot in the spatial transcriptomics data, each column represents a cell type from the single-cell RNA-seq data, and each entry indicates the proportion of a particular cell type in a spot. Additionally, we can obtain the anndata object of the spatial transcriptomics data with the deconvolution results embedded in the `.obs` attribute.

In [ ]:
pre = spoint_model.deconv_spatial()

In [ ]:
st_ad = spoint_model.st_ad

We can see the result of deconvolution in the ST slice.

In [ ]:
# sc.pl.embedding(st_ad,color=pre.columns[:20],basis='spatial',ncols=5)

In [ ]:
import pandas as pd

# 将 adata.obs 转换为 DataFrame
obs_df = st_ad.obs

# 选择需要写入文件的列
columns_to_save = ['Astro', 'Endo', 'Excitatory L2/3', 'Excitatory L4', 'Excitatory L5', 'Excitatory L6',
                   'Inhibitory Other', 'Inhibitory Pvalb', 'Inhibitory Sst', 'Inhibitory Vip', 'Micro',
                   'Neuron Other', 'Olig', 'Other', 'Smc']
0
# 确保这些列在 adata.obs 中存在
obs_df = obs_df[columns_to_save]

# 将 DataFrame 写入文本文件
output_file_path = '/data/SPACEL-l/docs/starmap2/SPACEL_result15.txt'
obs_df.to_csv(output_file_path, sep='\t', index=False)

print(f"数据已成功写入 {output_file_path}")


数据已成功写入 /data/SPACEL-l/docs/starmap2/SPACEL_result14.txt
